# Expérience 1, 2 et 3 avec les embeddings de NOUN

In [1]:
import numpy as np
from scipy import stats, spatial

In [2]:
folderPath = "embdFiles/"
nf_file = 'not_fake_'
f_file = "fake_"
endN = "_NOUN.txt"
listNouns = ['article','beard','blood','company','death','gun','id','interview','passport']

In [3]:
#here is a test to see if it is possible compute easily with numpy the average point for one dataset
path = folderPath + f_file + 'gun' + endN
arr = np.genfromtxt(path,delimiter=' ',dtype='float32')
avg = np.average(arr,axis=0)
avg.shape
#...of course it is possible, numpy can do everything

(1024,)

#### Pour chaque nom, on recupère les datasets avec et sans fake devant le nom dans un numpy array, et on stocke tout dans une liste

In [4]:
arrF_list = []
arrNF_list = []
for noun in listNouns:
    fPath = folderPath + f_file + noun + endN
    nfPath = folderPath + nf_file + noun + endN
    arrF = np.genfromtxt(fPath,delimiter=' ',dtype='float32')
    arrNF = np.genfromtxt(nfPath,delimiter=' ',dtype='float32')
    arrF_list.append(arrF)
    arrNF_list.append(arrNF)
    print("done for "+noun)

done for article
done for beard
done for blood
done for company
done for death
done for gun
done for id
done for interview
done for passport


#### Pour chaque nom, on calcule le point moyen, dans les deux cas. On stocke les points moyens dans des arrays

In [5]:
avgX_F = np.zeros((9,1024),dtype='float32')
avgX_NF = np.zeros((9,1024),dtype='float32')
avgX_F

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
for i in range(9):
    avgF = np.average(arrF_list[i],axis=0)
    avgNF = np.average(arrNF_list[i],axis=0)
    avgX_F[i] = avgF
    avgX_NF[i] = avgNF

In [7]:
avgX_F

array([[-0.04823099, -0.14354958, -0.01562697, ...,  0.01719081,
        -0.9936313 , -0.28639677],
       [-0.23168902, -0.30192888,  0.55367124, ...,  0.28872445,
        -0.15094164, -0.05723162],
       [ 0.23994008,  0.46857274,  0.6180047 , ..., -1.0538108 ,
         0.0454229 ,  0.0396353 ],
       ...,
       [-0.13170321, -0.27453583,  0.9930988 , ..., -0.5841659 ,
        -0.19219185, -0.31720725],
       [-0.16478896,  0.54115975, -0.07053245, ..., -0.5747151 ,
        -1.0550352 , -1.4621655 ],
       [-0.08293346, -0.015929  ,  0.19417278, ..., -0.5316996 ,
        -0.3460909 , -0.02548151]], dtype=float32)

#### Pour chaque nom, on récupère la liste des similarités cosinus entre le point moyen et chacun des points du dataset , et cela pour chacun des deux cas

In [8]:
simf_list = []
for i in range(9):
    avg = avgX_F[i]
    data = arrF_list[i]
    n = len(data)
    sim = np.zeros((n),dtype='float64')
    for j in range(n):
        sim[j] = 1-spatial.distance.cosine(data[j],avg) #1-distance pour avoir similarite
    simf_list.append(sim)

In [9]:
simnf_list = []
for i in range(9):
    avg = avgX_NF[i]
    data = arrNF_list[i]
    n = len(data)
    sim = np.zeros((n),dtype='float64')
    for j in range(n):
        sim[j] = 1-spatial.distance.cosine(data[j],avg) #1-distance pour avoir similarite
    simnf_list.append(sim)

# Expérience 1 : étude de dispersion des embeddings de NOUN selon la présence de FAKE devant

#### On cherche ici a montrer si Fake a une influence ou pas sur la dispersion des points autour du point moyen. On va regarder la moyenne des similarités cosinus pour l'ensemble des noms, dans les deux cas.

Commençons par construire deux échantillons distincts, selon si le nom est précédé ou non de FAKE

In [10]:
fake = np.concatenate(simf_list)
not_fake = np.concatenate(simnf_list)

Puis calculons les moyennes pour les deux échantillons

In [11]:
Sf = np.average(fake)
Snf = np.average(not_fake)

In [12]:
print(Sf,Snf)

0.8427543417241219 0.7565697389234177


In [16]:
fake.shape

(599,)

#### On constate que la présence de Fake devant le nom augmente la similiarité avec le point moyen. Mais cette différence entre les deux cas est-elle significative ? 

Verifions l'égalité des variances des deux échantillons

In [17]:
np.var(fake)/np.var(not_fake)

0.5633381012238198

Le rapport des deux variances est supérieur à 1/2 et inférieur à 2, on peut considérer l'égalité des variances. Passons maintenant au test de Student d'égalité des moyennes

In [18]:
stats.ttest_ind(fake,not_fake)

Ttest_indResult(statistic=21.488536412434197, pvalue=3.667909072774876e-102)

La p-value est largement inférieure au risque de premier espèce qu'on fixe à 5%. Donc on peut considérer qu'il y a une différence **significative** entre les moyennes des deux échantillons

#### Cette première expérience permet donc de conclure que la présence de Fake devant le nom diminue la dispersion des embeddings du nom. 

# Expérience 2 : comparaison des similitudes entre les embeddings de NOUN précédé et non précédé par FAKE, selon l'usage sémantique de FAKE

#### On cherche à savoir si l'usage de FAKE a un impact sur la similarité entre le point moyen des embeddings de NOUN avec FAKE et le point moyen des embeddings sans FAKE.

Commençons par définir les indexs des 3 principales catégories d'usage FAKE selon le nom auquel il est associé

In [19]:
#['article','beard','blood','company','death','gun','id','interview','passport']
inds_privatif = [1,2,3,4,5] #associé avec ces noms, FAKE prend un usage privatif
inds_ambivalent = [6,8] #associé avec ces noms, FAKE prend un usage ambivalent
inds_nonPrivatif = [0,7] #associé avec ces noms, FAKE prend un usage non-privatif

Les points moyens des embeddings ont déjà été calculés et stockés dans avgX_F et avgX_NF. Il ne reste qu'à calculer la similarité moyenne dans chacune des 3 catégories

In [20]:
priv = np.zeros((len(inds_privatif)),dtype='float64')
ambi = np.zeros((len(inds_ambivalent)),dtype='float64')
nonp = np.zeros((len(inds_nonPrivatif)),dtype='float64')
for i in range(len(priv)):
    ind = inds_privatif[i]
    priv[i] = 1- spatial.distance.cosine(avgX_F[ind],avgX_NF[ind])
for i in range(len(ambi)):
    ind = inds_ambivalent[i]
    ambi[i] = 1- spatial.distance.cosine(avgX_F[ind],avgX_NF[ind])
for i in range(len(nonp)):
    ind = inds_nonPrivatif[i]
    nonp[i] = 1- spatial.distance.cosine(avgX_F[ind],avgX_NF[ind])

In [21]:
Spriv = np.mean(priv)
Sambi = np.mean(ambi)
Snonp = np.mean(nonp)

In [22]:
print(Spriv,Sambi,Snonp)

0.9078827977180481 0.9133485555648804 0.8935658931732178


In [46]:
len(np.concatenate([arrF_list[i] for i in inds_nonPrivatif]))

20525

In [52]:
len(arrNF_list[0])

10392

Selon l'hypothèse linguistique, on a supposé que le nom précédé de FAKE lorsqu'il était utilisé dans un usage non-privatif resterait davantage similaire au nom sans FAKE que lorsque FAKE était utilisé dans un usage ambivalent, et que l'usage ambivalent éloignait lui-même les deux formes du nom dans une moindre mesure que lorsque FAKE était employé privativement. En d'autres termes, on aurait supposé *Spriv < Sambi < Snonp*

On constate ici que **ce n'est pas le cas**. L'usage de FAKE ne semble pas avoir d'influence sur l'éloignement des deux formes de NOUN avec ou sans FAKE. <br/><br/> **Notre hypothèse est donc rejetée**

# Expérience 3 : étude de l'influence de l'usage de FAKE sur la variation de dispersion des embeddings de NOUN

On a montré dans l'expérience 1 que la présence de FAKE avant le nom réduisait la dispersion des embeddings de NOUN autour du point moyen. Maintenant, on se demande si la catégorie d'usage de FAKE a une influence sur la variation de dispersion.

In [27]:
df_priv = np.concatenate([simf_list[i] for i in inds_privatif])
df_ambi = np.concatenate([simf_list[i] for i in inds_ambivalent])
df_nonp = np.concatenate([simf_list[i] for i in inds_nonPrivatif])
dnf_priv = np.concatenate([simnf_list[i] for i in inds_privatif])
dnf_ambi = np.concatenate([simnf_list[i] for i in inds_ambivalent])
dnf_nonp = np.concatenate([simnf_list[i] for i in inds_nonPrivatif])

On définit une fonction pour réaliser les Tests d'égalité de moyennes. On va en effet commencer par vérifier que, dans chaque catégorie, la présence de FAKE modifie bien la dispersion.

In [28]:
def testEgalMoyenne(arr1,arr2,alpha = 1e-3):
    '''
    Retourne False si la p-value du test de Student d'égalité de moyenne est inférieure au risque de premiere espece.
    Cela signifie qu'il n'y a pas égalité des moyennes.
    Sinon, retourne True et on ne peut rejeter l'hypothèse d'égalité des moyennes
    '''
    r = np.var(arr1)/np.var(arr2)
    if r > 0.5 and r < 2 :
        #pas d'egalite des variances
        p = stats.ttest_ind(arr1,arr2,equal_var=True)[1]
    else:
        #pas d'egalite des variances
        p = stats.ttest_ind(arr1,arr2,equal_var=False)[1]
    if p > alpha:
        return True
    else:
        return False

In [29]:
print('\npriv : ',testEgalMoyenne(df_priv,dnf_priv))
print('\nambi : ',testEgalMoyenne(df_ambi,dnf_ambi))
print('\nnonp : ',testEgalMoyenne(df_nonp,dnf_nonp))
#Si le test affiche FALSE c'est que les moyennes sont significativement différentes


priv :  False

ambi :  False

nonp :  False


Cela ne fait que confirmer le résultat de l'expérience 1 appliqué cette fois uniquement au sein de chaque catégorie. Cela signifie que l'ont peut établir pour chaque catégorie un coefficient supérieur à 1 par lequel la similarité est multipliée lorsque le nom est précédé de FAKE.

In [30]:
coeff_priv = np.mean(df_priv)/np.mean(dnf_priv)
coeff_ambi = np.mean(df_ambi)/np.mean(dnf_ambi)
coeff_nonp = np.mean(df_nonp)/np.mean(dnf_nonp)

A l'instar de l'hypothèse formulée lors de l'expérience 2, on pourrait formuler l'hypothèse suivante : <br/> L'usage de FAKE fait varier le coefficient de multiplication de la dispersion des embeddings de NOUN avec et sans FAKE. Soit (1) la privativité de FAKE augmente ce coefficient (c'est à dire que les embeddings sont dautant plus similaires que FAKE est privatif), soit (2) la privativité de FAKE fait diminuer ce coefficient (auquel cas la privativité de FAKE disperse davantage les embeddings). Autrement dit, on s'attend à : <br/> 1 - soit coeff_priv > coeff_ambi > coeff_nonp <br/> 2 - soit coeff_priv < coeff_ambi < coeff_nonp

In [31]:
print(coeff_priv,coeff_ambi,coeff_nonp)

1.1251746765628374 1.1401544722178965 1.0908976907793215


In [34]:
np.mean(df_priv)-np.mean(dnf_priv),np.mean(df_ambi)-np.mean(dnf_ambi),np.mean(df_nonp)-np.mean(dnf_nonp)

(0.09427108888349034, 0.10070390364848869, 0.07269134883473438)

#### On constate ici que de telles relations d'ordres n'existent pas sur les variations de dispersion des embeddings de NOUN. On ne peut donc pas conclure à l'influence du degré de privativité de FAKE sur la variation montrée à l'expérience 1.